In [1]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
import traceback
import sys

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")


In [2]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
DF_info = pd.read_csv(f"{path_work}/train_nn/TropiGATv2.final_df.tsv", sep = "\t" ,  header = 0)
DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_lvl_0 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

# Log file : 
path_ensemble = f"{path_work}/train_nn/ensemble_tailored"
# os.mkdir(path_ensemble)

df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))


FileNotFoundError: [Errno 2] No such file or directory: '/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/TropiGATv2.final_df.tsv'

In [ ]:
# *****************************************************************************
#*****  Make graphs : 
# Tailored parameters :
tailored_kltype = ""
ratio = 7
training_frac , test_frac , eval_frac = 0.7 , 0.2 , 0.1
model_version = "small"

graph_baseline , dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(DF_info_lvl_0)
graph_dico = {tailored_kltype : TropiGAT_graph.build_graph_masking(graph_baseline ,dico_prophage_kltype_associated, DF_info_lvl_0, tailored_kltype, ratio , training_frac , test_frac , eval_frac)}
version_dico = {"big" : TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1), 
                "small" : TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)}


> Training

In [ ]:
# *****************************************************************************
def train_graph(KL_type) :
    with open(f"{path_work}/train_nn/ensemble_tailored/{KL_type}__node_classification.0612.log" , "w") as log_outfile :
        # calling the relevatn objects :
        n_prophage = dico_prophage_count[KL_type]
        graph_data_kltype = graph_dico[KL_type]
        model = version_dico[model_version]
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
        scheduler = ReduceLROnPlateau(optimizer, 'min')
        criterion = torch.nn.BCEWithLogitsLoss()
        early_stopping = TropiGAT_models.EarlyStopping(patience=60, verbose=True, path=f"{path_ensemble}/{KL_type}.TropiGATv2.0612.pt", metric='MCC')
        # initialize the model :
        model(graph_data_kltype)
        try : 
            for epoch in range(200):
                train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer,criterion)
                if epoch % 5 == 0:
                    # Get all metrics
                    test_loss, metrics, _ = TropiGAT_models.evaluate(model, graph_data_kltype,criterion, graph_data_kltype["B1"].test_mask)
                    info_training_concise = f'Epoch: {epoch}\tTrain Loss: {train_loss}\tTest Loss: {test_loss}\tMCC: {metrics[3]}\tAUC: {metrics[5]}\tAccuracy: {metrics[4]}\n'
                    info_training = f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss},F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                    log_outfile.write(info_training_concise)
                    scheduler.step(test_loss)
                    early_stopping(metrics[3], model, epoch)
                    if early_stopping.early_stop:
                        log_outfile.write(f"Early stopping at epoch = {epoch}\n")
                        break
            else :
                torch.save(model, f"{path_ensemble}/{KL_type}.TropiGATv2.0612.pt")
            # The final eval :
            # Load the saved model :
            model_final = version_dico[model_version]
            model_final.load_state_dict(torch.load(f"{path_ensemble}/{KL_type}.TropiGATv2.0612.pt"))
            eval_loss, metrics , weights = TropiGAT_models.evaluate(model_final, graph_data_kltype, criterion,graph_data_kltype["B1"].eval_mask)
            # Write the tailoring results : 
            with open(f"{path_ensemble}/Metric_Report_Tailoring.0612.tsv", "a+") as metric_outfile :
                metric_outfile.write(f"{KL_type}\t{n_prophage}\t{metrics[0]}\t{metrics[1]}\t{metrics[2]}\t{metrics[3]}\t{metrics[4]}\t{metrics[5]}\t{ratio},{training_frac},{test_frac},{eval_frac}\n")
            info_eval = f'Epoch: {epoch}, F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
            log_outfile.write(f"Final evaluation ...\n{info_eval}")
            with open(f"{path_ensemble}/Weights_Report_Tailoring.{KL_type}.0612.tsv", "a+") as weight_outfile :
                weight_outfile.write(f"{str(weights)}")
        except Exception as e :
            exc_type, exc_value, exc_traceback = sys.exc_info()
            tb_lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            log_outfile.write(f"***Issue here : {'\n'.join({tb_lines})")
            with open(f"{path_ensemble}/Metric_Report.0612.tsv", "a+") as metric_outfile :
                n_prophage = dico_prophage_count[KL_type]
                metric_outfile.write(f"{KL_type}\t{n_prophage}\t***Issue***\n")

if __name__ == '__main__':
    train_graph(tailored_kltype)


> Optuna

In [ ]:
# Version here :